<a href="https://colab.research.google.com/github/q890003/Hybrid-image_Image-pyramid_Colorizing-the-Russian-Empire/blob/master/ML_hw4_EM_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import dill
import numpy as np
import math
from itertools import product 
from struct import unpack 

def get_img(img_file_path):
    with open(img_file_path, "rb") as img_file:
        magic, num_imgs, rows, cols = unpack(">IIII", img_file.read(16))

        imgs_mtx = np.zeros((num_imgs, rows, cols), dtype=int)
        for img in range(num_imgs):
            for (row, col) in product(range(rows), range(cols)):
                tmp = unpack(">B", img_file.read(1))
                tmp = int(str(tmp[0]))
                imgs_mtx[img][row,col] = tmp
    return imgs_mtx, num_imgs

def get_label(lbl_file_path):
    with open(lbl_file_path, "rb") as label_file:
        magic, num_itms = unpack(">II", label_file.read(8))
        lbls_mtx = np.zeros((num_itms), dtype=int)
        for img in range(num_itms):
            label = unpack(">B", label_file.read(1))
            label = int(str(label[0]))
            lbls_mtx[img] = label
    return lbls_mtx

label_types = 10
gray_bins = 2
#mode = int(sys.argv[1])  #mode = 0 -> Discrete; mode = 1 -> Contineous
mode = 0    # for test
rows = 28
cols = 28

train_img_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-images.idx3-ubyte"
train_label_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw4/train-labels.idx1-ubyte"
train_img_mtx, train_num_imgs = get_img(train_img_pth)
train_label_mtx = get_label(train_label_pth)

# test_img_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw2/t10k-images.idx3-ubyte"
# test_label_pth = "/content/drive/My Drive/Colab Notebooks/ML_hw2/t10k-labels.idx1-ubyte"
# test_img_mtx, tests_num_imgs = get_img(test_img_pth)
# test_label_mtx = get_label(test_label_pth)


labels_count = np.zeros((label_types), dtype=float)
likelyhood = np.zeros((label_types,rows,cols,gray_bins), dtype=float)
for img in range(train_num_imgs):
    lbl = train_label_mtx[img]
    labels_count[lbl] += 1
    for (row, col) in product(range(rows), range(cols)):
        train_img_mtx[img][row, col] = train_img_mtx[img][row, col] // 128
        gray = train_img_mtx[img][row, col]
        likelyhood[lbl][row,col,gray] += 1
prior = labels_count/train_num_imgs







In [0]:
#validation
true = np.zeros(10, dtype=float)
for img in range(train_num_imgs):
    true[train_label_mtx[img]] += 1


In [0]:
# init
q = np.full((10,28,28), 0.1)  #under 28*28 pixels, prob of 0~9 init is 0.1,...,0.1
p = np.full((10,28,28), 0.5)  #under 28*28 pixels, a probality of bin=1  in each number, 0~9 
w_1 = np.zeros((10,28,28), dtype=float)  #under 28*28 pixels, When a pixel has bin = 1, the prob of a number under numbers 0~9 
w_0 = np.zeros((10,28,28), dtype=float)  #under 28*28 pixels, When a pixel has bin = 0, the prob of a number under numbers 0~9

for iter in range(2):
    # Expectation step:


    #weight in a image.
    # expect w_1, w_0.
    w_1_sum = np.zeros((28,28), dtype=float)
    w_0_sum = np.zeros((28,28), dtype=float)

    #q_i * p_i in each pixel.
    w_1_pxls = q * p     
    w_0_pxls = q * (1- p)

    for i in range(10):
        w_1_sum += w_1_pxls[i]
        w_0_sum += w_0_pxls[i]
    # w_i = (q_i * p_i) / sum(q_i * p_i)
    for i in range(10):
        w_1[i] = np.true_divide(q[i]*    p[i]   , w_1_sum)
        w_0[i] = np.true_divide(q[i] *(1 - p[i]), w_0_sum)

    print("@@@@@@@@@@@@@@@@@iter", iter, "@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
    print(q[0],"q[0")
    print(w_1[0], "w_1[0]")
    # Maximization step:
    # update q, p
    for i in range(10): #class 0 to 9
        tmp = np.zeros((28,28), dtype=float)
        count_1 = np.zeros((28,28), dtype=float)
        for img in range(train_num_imgs):
            tmp += w_1[i] * train_img_mtx[img] + w_0[i] * (1 - train_img_mtx[img])
            count_1 += train_img_mtx[img]
        # print(tmp)
        q[i] = tmp / train_num_imgs
        p[i] = np.true_divide( count_1 * w_1[i] , count_1 * w_1[i] + (train_num_imgs - count_1)* w_0[i] )
    print("!!!!!!!!!!!!!!!!!!! Update @!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(q[0], "q[0")
    print(p[0], "p[0]")


In [63]:
#clustering
cluster = np.zeros((10,28,28), dtype=float) 
predict = np.zeros(10, dtype=float) 
record = np.zeros(10, dtype=float)

for img in range(train_num_imgs):
    for i in range (1):
        cluster[i] = np.log(w_1[i]* train_img_mtx[img] + w_0[i]* (1- train_img_mtx[img]))
        predict[i] = np.sum(cluster[i])
    label = np.max(predict)
    record[int(label)] += 1

print("@@@@@@@")

print(record)
print(true)

ValueError: ignored